In [1]:
import numpy as np

# Load the sample data into NumPy arrays
features = np.load("../generated_games/features-40k.npy")
labels = np.load("../generated_games/labels-40k.npy")

print("features", features.shape)
print("labels", labels.shape)

features (41439, 1, 9, 9)
labels (41439, 81)


In [2]:
# display a matrix of integers
def print_matrix(matrix, cell_width=3):
    for row in matrix:
        print(" ".join(f"{int(cell):{cell_width}}" for cell in row))

In [3]:
# By setting a random seed, you make sure this script is exactly reproducible.
np.random.seed(123)

X = features
Y = labels
samples = X.shape[0]
board_size = 9 * 9
X = X.reshape(samples, board_size)
Y = Y.reshape(samples, board_size)
print("X[20]")
print_matrix(X[20].reshape(9, 9))
print("Y[20]")
print_matrix(Y[20].reshape(9, 9))

# Transform the input into vectors of size 81, instead of 9 × 9 matrices.
# Hold back 10% of the data for a test set; train on the other 90%.
train_samples = int(0.9 * samples)
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

X[20]
  0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0
  0   0   0  -1  -1  -1   1   0   0
  1  -1  -1   0  -1   1   1  -1   0
  0   1   1   1   1  -1   1   0   0
  0   0   0   1  -1  -1   0   1   0
  0  -1   0   1   1  -1   0   0   0
  0   0   0  -1  -1   1   0   0   0
  0   0   0   0   0   0   0   0   0
Y[20]
  0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   1   0
  0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0
X_train (37295, 81)
Y_train (37295, 81)
X_test (4144, 81)
Y_test (4144, 81)


In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
import time

model = Sequential()
model.add(Input(shape=(board_size,)))
model.add(Dense(1000, activation='sigmoid'))
model.add(Dense(500, activation='sigmoid'))
model.add(Dense(board_size, activation='sigmoid'))
model.summary()

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

# Record the start time
start_time = time.time()
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=15,
          verbose=1,
          validation_data=(X_test, Y_test))
# Record the end time
end_time = time.time()
# Calculate and display the wall time
print(f"Wall time: {end_time - start_time:.6f} seconds")

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2024-11-24 00:41:30.542549: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 00:41:30.548153: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 00:41:30.572232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732437690.615201    3641 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732437690.624299    3641 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 00:41:30.653793: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1000)           │        82,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 500)            │       500,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 81)             │        40,581 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 623,081 (2.38 MB)

 Trainable params: 623,081 (2.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0133 - loss: 0.1842 - val_accuracy: 0.0130 - val_loss: 0.0476
Epoch 2/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0117 - loss: 0.0394 - val_accuracy: 0.0128 - val_loss: 0.0256
Epoch 3/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0136 - loss: 0.0238 - val_accuracy: 0.0195 - val_loss: 0.0198
Epoch 4/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0171 - loss: 0.0190 - val_accuracy: 0.0246 - val_loss: 0.0173
Epoch 5/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0212 - loss: 0.0169 - val_accuracy: 0.0246 - val_loss: 0.0159
Epoch 6/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0229 - loss: 0.0157 - val_accuracy: 0.0241 - val_loss: 0.0151
Epoch 7/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.0217 - loss: 0.0149 - val_accuracy: 0.0246 - val_loss: 0.0145
Epoch 8/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0252 - loss: 0.0144 - val_accuracy: 0.

In [5]:
test_board = np.array([[
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 0,  1, -1,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
]])
move_probs = model.predict(test_board)[0]
i = 0
for row in range(9):
    row_formatted = []
    for col in range(9):
        row_formatted.append('{:.3f}'.format(move_probs[i]))
        i += 1
    print(' '.join(row_formatted))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
0.036 0.036 0.040 0.038 0.037 0.039 0.039 0.038 0.036
0.038 0.041 0.039 0.042 0.043 0.042 0.041 0.040 0.039
0.038 0.041 0.037 0.046 0.041 0.045 0.035 0.041 0.038
0.038 0.043 0.045 0.045 0.043 0.042 0.038 0.041 0.037
0.038 0.041 0.045 0.041 0.045 0.045 0.043 0.042 0.037
0.039 0.040 0.045 0.045 0.047 0.044 0.043 0.042 0.038
0.038 0.040 0.037 0.045 0.046 0.047 0.038 0.041 0.037
0.037 0.042 0.041 0.043 0.042 0.043 0.040 0.039 0.038
0.035 0.038 0.039 0.039 0.039 0.040 0.039 0.037 0.036


Starlight MacBook Air M2:
> Wall time: 32.514940 seconds

> Test loss: 0.012939339503645897
> Test accuracy: 0.02292471006512642

MacBookPro M1 Pro:
> Wall time: 40.835932 seconds

> Test loss: 0.01292334869503975
> Test accuracy: 0.028233591467142105

Seahawk Intel i7-4770 @ 3.4GHz:
> Wall time: 52.658726 seconds

> Test loss: 0.01294137816876173
> Test accuracy: 0.027027027681469917